## Implementing VGG16 - 21BAI1007 Goutham Krishnan

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [12]:
num_epochs = 1
batch_size = 40
learning_rate = 0.001
classes = ('plane', 'car' , 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [13]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
    )
])
train_dataset = torchvision.datasets.CIFAR10(root= './data', train = True, download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(root= './data', train = False, download =True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = True)
n_total_step = len(train_loader)
print(n_total_step)

1250


In [15]:
model = models.vgg16(pretrained = True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Linear(input_lastLayer,10)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)

In [16]:
for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)

        labels_hat = model(imgs)
        n_corrects = (labels_hat.argmax(axis=1) == labels).sum().item()
        loss_value = criterion(labels_hat, labels)

        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (i + 1) % 250 == 0:
            print(f'epoch {epoch + 1}/{num_epochs}, step: {i + 1}/{len(train_loader)}: loss = {loss_value:.5f}, acc = {100 * (n_corrects / labels.size(0)):.2f}%')

    print()


epoch 1/1, step: 250/1250: loss = 0.41883, acc = 82.50%
epoch 1/1, step: 500/1250: loss = 0.69434, acc = 70.00%
epoch 1/1, step: 750/1250: loss = 0.48321, acc = 87.50%
epoch 1/1, step: 1000/1250: loss = 0.40966, acc = 82.50%
epoch 1/1, step: 1250/1250: loss = 0.10697, acc = 97.50%



In [17]:
with torch.no_grad():
    number_corrects = 0
    number_samples = 0
    for i, (test_images_set , test_labels_set) in enumerate(test_loader):
        test_images_set = test_images_set.to(device)
        test_labels_set = test_labels_set.to(device)

        y_predicted = model(test_images_set)
        labels_predicted = y_predicted.argmax(axis = 1)
        number_corrects += (labels_predicted==test_labels_set).sum().item()
        number_samples += test_labels_set.size(0)
    print(f'Overall accuracy {(number_corrects / number_samples)*100}%')

Overall accuracy 89.64999999999999%
